# AWS Sagemaker: Bring Your Own Model


This notebook follows this [medium post](https://medium.com/p/92778dabb32f) on how to bring your own model into AWS SageMaker.

It outlines how you can package any machine learning algorithm into a docker container and then ship it to the cloud.

This example uses the [Keras](https://keras.io/) but you could use any library you like.

## A trained model

Let's get a trained model. Here is an example which uses [ResNet50](https://arxiv.org/abs/1512.03385) trained on ImageNet. It is one of the Keras [available models](https://keras.io/applications/#available-models)

In [4]:
!python get_model.py

Using TensorFlow backend.
2018-07-27 11:40:23.954194: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA


This will download *model.json* and *weights.hdf5*. These files represent the model architecture and trained weights, respectively. When you are ready you can replace them with your own trained model. Here is where they go.

In [5]:
!ls ./container/model

model.json  weights.hdf5


## Build Docker image

You will need [Docker](https://www.docker.com/) to run this notebook. 

**Docker Installation**

To install Docker, you can use the Docker Toolbox for [Windows](https://docs.docker.com/toolbox/toolbox_install_windows/) and [Mac](https://docs.docker.com/toolbox/toolbox_install_mac/). For linux just run this command

In [8]:
# uncomment for linux installation (if required)
# !wget -qO- https://get.docker.com/ | sh

**Docker Build**

Read the Dockerfile and build a docker image

In [9]:
!image=my_model:v1.0.0
!docker build ./container --tag $(image)

Sending build context to Docker daemon 103.1 MB
Step 1/12 : FROM ubuntu:16.04
 ---> e13f3d529b1a
Step 2/12 : MAINTAINER Sam Murphy <samsammurphy@gmail.com>
 ---> Running in 6fd653a13966
 ---> d10b50a28ade
Removing intermediate container 6fd653a13966
Step 3/12 : EXPOSE 8080
 ---> Running in 3ecdc890d075
 ---> 167804be6ff0
Removing intermediate container 3ecdc890d075
Step 4/12 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          python          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Running in 93aebb87a6b2
Get:1 http://security.ubuntu.com/ubuntu xenial-security InRelease [107 kB]
Get:2 http://archive.ubuntu.com/ubuntu xenial InRelease [247 kB]
Get:3 http://archive.ubuntu.com/ubuntu xenial-updates InRelease [109 kB]
Get:4 http://archive.ubuntu.com/ubuntu xenial-backports InRelease [107 kB]
Get:5 http://security.ubuntu.com/ubuntu xenial-security/universe Sources [84.7 kB]
Get:6 http://security.ubuntu.com/ubuntu xenial

debconf: delaying package configuration, since apt-utils is not installed
Fetched 19.1 MB in 2s (6522 kB/s)
Selecting previously unselected package libxau6:amd64.
(Reading database ... 4768 files and directories currently installed.)
Preparing to unpack .../libxau6_1%3a1.0.8-1_amd64.deb ...
Unpacking libxau6:amd64 (1:1.0.8-1) ...
Selecting previously unselected package libjpeg-turbo8:amd64.
Preparing to unpack .../libjpeg-turbo8_1.4.2-0ubuntu3.1_amd64.deb ...
Unpacking libjpeg-turbo8:amd64 (1.4.2-0ubuntu3.1) ...
Selecting previously unselected package libpython2.7-minimal:amd64.
Preparing to unpack .../libpython2.7-minimal_2.7.12-1ubuntu0~16.04.3_amd64.deb ...
Unpacking libpython2.7-minimal:amd64 (2.7.12-1ubuntu0~16.04.3) ...
Selecting previously unselected package python2.7-minimal.
Preparing to unpack .../python2.7-minimal_2.7.12-1ubuntu0~16.04.3_amd64.deb ...
Unpacking python2.7-minimal (2.7.12-1ubuntu0~16.04.3) ...
Selecting previously unselected package python-minimal.
Preparing t

Setting up libpython-stdlib:amd64 (2.7.12-1~16.04) ...
Setting up python (2.7.12-1~16.04) ...
Setting up libjbig0:amd64 (2.1-3.1) ...
Setting up libidn11:amd64 (1.32-3ubuntu1.2) ...
Setting up libpng12-0:amd64 (1.2.54-1ubuntu1.1) ...
Setting up ucf (3.0036) ...
debconf: unable to initialize frontend: Dialog
debconf: (TERM is not set, so the dialog frontend is not usable.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (Can't locate Term/ReadLine.pm in @INC (you may need to install the Term::ReadLine module) (@INC contains: /etc/perl /usr/local/lib/x86_64-linux-gnu/perl/5.22.1 /usr/local/share/perl/5.22.1 /usr/lib/x86_64-linux-gnu/perl5/5.22 /usr/share/perl5 /usr/lib/x86_64-linux-gnu/perl/5.22 /usr/share/perl/5.22 /usr/local/lib/site_perl /usr/lib/x86_64-linux-gnu/perl-base .) at /usr/share/perl5/Debconf/FrontEnd/Readline.pm line 7.)
debconf: falling back to frontend: Teletype
Setting up openssl (1.0.2g-1ubuntu4.13) ...
Setting up c

  Running setup.py bdist_wheel for itsdangerous: started
  Running setup.py bdist_wheel for itsdangerous: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/2c/4a/61/5599631c1554768c6290b08c02c72d7317910374ca602ff1e5
  Running setup.py bdist_wheel for MarkupSafe: started
  Running setup.py bdist_wheel for MarkupSafe: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/33/56/20/ebe49a5c612fffe1c5a632146b16596f9e64676768661e4e46
Successfully built itsdangerous MarkupSafe
 ---> 581048ad8a8f
Removing intermediate container 5d2c304f79df
Step 6/12 : ENV PYTHONUNBUFFERED TRUE
 ---> Running in 7c3230295f1d
 ---> dda3d275bebf
Removing intermediate container 7c3230295f1d
Step 7/12 : ENV PYTHONDONTWRITEBYTECODE TRUE
 ---> Running in 2a4a8f586300
 ---> adf2564f76cd
Removing intermediate container 2a4a8f586300
Step 8/12 : ENV PATH "/opt/program:${PATH}"
 ---> Running in c65b6495c526
 ---> 99b27dc50109
Removing intermediate container c65b6495c526
Step 9

  Running setup.py bdist_wheel for gast: started
  Running setup.py bdist_wheel for gast: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/9a/1f/0e/3cde98113222b853e98fc0a8e9924480a3e25f1b4008cedb4f
  Running setup.py bdist_wheel for absl-py: started
  Running setup.py bdist_wheel for absl-py: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/4c/16/ef/e36a23f2432e9220f8845f94e2c3abd39e7d9d1cd458d3159d
  Running setup.py bdist_wheel for termcolor: started
  Running setup.py bdist_wheel for termcolor: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
  Running setup.py bdist_wheel for pyyaml: started
  Running setup.py bdist_wheel for pyyaml: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/ad/da/0c/74eb680767247273e2cf2723482cb9c924fe70af57c334513f
Successfully built gast absl-py termcolor pyyaml
  Found existing installation: 

## Local test

Start a local webserver with the machine learning code and endpoints all set up

In [27]:
!./container/run_local_server.sh $(image)

613d91b848634e3fb5bce5649fe21c39fdf1a40f74e9680dff69c5bb9acb6a57


This endpoint checks if the machine learning algorithms loads successfully

In [23]:
!curl -X GET 'http://localhost:8080/ping'

loaded model successfully!

This command sends an image to the /predict endpoint, which will run the machine learning algorithm and return what it thinks is in the image.

In [24]:
!curl -X POST -F image=@$(pwd)/images/dog.jpg 'http://localhost:8080/predict'

{"predictions": [{"probability": 0.9901767373085022, "label": "beagle"}, {"probability": 0.0022487046662718058, "label": "Walker_hound"}, {"probability": 0.0011901347897946835, "label": "Brittany_spaniel"}, {"probability": 0.0011802910594269633, "label": "pot"}, {"probability": 0.0006831124192103744, "label": "Cardigan"}], "success": true}

ok, so we can identify that dog as being a beagle but can we determine if something is a hotdog?

In [28]:
!curl -X POST -F image=@$(pwd)/images/hotdog.jpg 'http://localhost:8080/predict'

{"predictions": [{"probability": 0.9936224818229675, "label": "hotdog"}, {"probability": 0.005856595002114773, "label": "cheeseburger"}, {"probability": 0.00016705217421986163, "label": "bakery"}, {"probability": 0.00010399545863037929, "label": "tennis_ball"}, {"probability": 9.467774361837655e-05, "label": "French_loaf"}], "success": true}

When you are done with the local test you can kill the container that is running in the background.

In [29]:
!docker kill mylocalservice

mylocalservice


## Register the container

Here is how to send the container in Amazon's Elastic Container Registery

In [ ]:
!./container/send_container_to_amazon.sh $(image)

# Part 2: Training and Hosting your Algorithm in Amazon SageMaker

Once you have your container packaged, you can use it to train and serve models. Let's do that with the algorithm we made above.

## Set up the environment

Here we specify a bucket to use and the role that will be used for working with SageMaker.

In [ ]:
# S3 prefix
prefix = 'DEMO-scikit-byo-iris'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

## Create the session

The session remembers our connection parameters to SageMaker. We'll use it to perform all of our SageMaker operations.

In [ ]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

## Deploy the model

Deploying the model to SageMaker hosting just requires a `deploy` call on the fitted model. This call takes an instance count, instance type, and optionally serializer and deserializer functions. These are used when the resulting predictor is created on the endpoint.

In [ ]:
from sagemaker.predictor import csv_serializer
predictor = tree.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer)

## Choose some data and use it for a prediction

In order to do some predictions, we'll extract some of the data we used for training and do predictions against it. This is, of course, bad statistical practice, but a good way to see how the mechanism works.

In [ ]:
shape=pd.read_csv("data/iris.csv", header=None)

import itertools

a = [50*i for i in range(3)]
b = [40+i for i in range(10)]
indices = [i+j for i,j in itertools.product(a,b)]

test_data=shape.iloc[indices[:-1]]
test_X=test_data.iloc[:,1:]
test_y=test_data.iloc[:,0]

Prediction is as easy as calling predict with the predictor we got back from deploy and the data we want to do predictions with. The serializers take care of doing the data conversions for us.

In [ ]:
print(predictor.predict(test_X.values).decode('utf-8'))

## Optional cleanup

When you're done with the endpoint, you'll want to clean it up.

In [ ]:
sess.delete_endpoint(predictor.endpoint)

In [ ]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/decision-trees-sample:latest'.format(account, region)

tree = sage.estimator.Estimator(image,
                       role, 1, 'ml.c4.2xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)

tree.fit(data_location)

## Create an estimator and fit the model

In order to use SageMaker to fit our algorithm, we'll create an `Estimator` that defines how to use the container to train. This includes the configuration we need to invoke SageMaker training:

* The __container name__. This is constructed as in the shell commands above.
* The __role__. As defined above.
* The __instance count__ which is the number of machines to use for training.
* The __instance type__ which is the type of machine to use for training.
* The __output path__ determines where the model artifact will be written.
* The __session__ is the SageMaker session object that we defined above.

Then we use fit() on the estimator to train against the data that we uploaded above.

In [ ]:
WORK_DIRECTORY = 'data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

## Upload the data for training

When training large models with huge amounts of data, you'll typically use big data tools, like Amazon Athena, AWS Glue, or Amazon EMR, to create your data in S3. For the purposes of this example, we're using some the classic [Iris dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set), which we have included. 

We can use use the tools provided by the SageMaker Python SDK to upload the data to a default bucket. 